In [24]:
#import Dependencies
import pandas as pd
import dateutil.parser
from datetime import datetime
import numpy as np
import hvplot.pandas
import json
import requests
import matplotlib.pyplot as plt
import time
from census import Census
import warnings
warnings.filterwarnings("ignore")

In [25]:
nyc_url = "https://data.cityofnewyork.us/resource/8h9b-rp9u.json?$where=arrest_date between '2017-01-01T00:00:00' and '2023-12-31T23:59:59'&$limit=1000000"
nyc_response= requests.get(nyc_url).json()
nyc_df=pd.DataFrame(nyc_response)


In [26]:
nyc_df.head()

,arrest_key,arrest_date,pd_cd,pd_desc,ky_cd,ofns_desc,law_code,law_cat_cd,arrest_boro,arrest_precinct,jurisdiction_code,age_group,perp_sex,perp_race,x_coord_cd,y_coord_cd,latitude,longitude,lon_lat
0,159838726,2017-01-01T00:00:00.000,115.0,RECKLESS ENDANGERMENT 2,355.0,OFFENSES AGAINST THE PERSON,PL 1202000,M,M,14,0.0,18-24,M,WHITE,988912.0,212647.0,40.750350440000034,-73.98317545899994,"{'type': 'Point', 'coordinates': [-73.98317545..."
1,159824786,2017-01-01T00:00:00.000,101.0,ASSAULT 3,344.0,ASSAULT 3 & RELATED OFFENSES,PL 1200001,M,M,18,0.0,25-44,M,WHITE,992043.0,217246.0,40.76297132000008,-73.97186979099996,"{'type': 'Point', 'coordinates': [-73.97186979..."
2,159840237,2017-01-01T00:00:00.000,729.0,"FORGERY,ETC.,UNCLASSIFIED-FELONY",113.0,FORGERY,PL 1702500,F,B,40,0.0,25-44,M,WHITE HISPANIC,1006669.0,233631.0,40.80791877300004,-73.91901728199997,"{'type': 'Point', 'coordinates': [-73.91901728..."
3,159828861,2017-01-01T00:00:00.000,744.0,BAIL JUMPING 3,359.0,OFFENSES AGAINST PUBLIC ADMINISTRATION,PL 2155500,M,K,67,0.0,25-44,M,BLACK,1009039.0,176368.0,40.650739599000076,-73.91066707899995,"{'type': 'Point', 'coordinates': [-73.91066707..."
4,159834533,2017-01-01T00:00:00.000,203.0,"TRESPASS 3, CRIMINAL",352.0,CRIMINAL TRESPASS,PL 1401000,M,K,60,0.0,18-24,M,BLACK,986229.0,148311.0,40.573763375000055,-73.99287775699997,"{'type': 'Point', 'coordinates': [-73.99287775..."


In [5]:
# nyc_df.columns

# columns = ['arrest_key', 'arrest_date', 'ofns_desc',
#        'law_cat_cd', 'perp_race', 'x_coord_cd',
#        'y_coord_cd', 'latitude', 'longitude']

# nyc_df = nyc_df[columns]
# nyc_df.head()

In [27]:
nyc_df = nyc_df.rename(
    columns= {
        'arrest_key': 'Complaint ID',
        'arrest_date' : 'Date',
        'ofns_desc' : 'Complaint Description',
       'law_cat_cd': 'Sex',
        'perp_race' : 'Race',
        'x_coord_cd': 'x_coord',
       'y_coord_cd': 'y_coord',
        'latitude': 'Lat',
        'longitude' : 'Lng'
    })
nyc_df["NYC"]= "NYC"
nyc_df["Crime"]= "Crime"
nyc_df.head()

,Complaint ID,Date,pd_cd,pd_desc,ky_cd,Complaint Description,law_code,Sex,arrest_boro,arrest_precinct,...,age_group,perp_sex,Race,x_coord,y_coord,Lat,Lng,lon_lat,NYC,Crime
0,159838726,2017-01-01T00:00:00.000,115.0,RECKLESS ENDANGERMENT 2,355.0,OFFENSES AGAINST THE PERSON,PL 1202000,M,M,14,...,18-24,M,WHITE,988912.0,212647.0,40.750350440000034,-73.98317545899994,"{'type': 'Point', 'coordinates': [-73.98317545...",NYC,Crime
1,159824786,2017-01-01T00:00:00.000,101.0,ASSAULT 3,344.0,ASSAULT 3 & RELATED OFFENSES,PL 1200001,M,M,18,...,25-44,M,WHITE,992043.0,217246.0,40.76297132000008,-73.97186979099996,"{'type': 'Point', 'coordinates': [-73.97186979...",NYC,Crime
2,159840237,2017-01-01T00:00:00.000,729.0,"FORGERY,ETC.,UNCLASSIFIED-FELONY",113.0,FORGERY,PL 1702500,F,B,40,...,25-44,M,WHITE HISPANIC,1006669.0,233631.0,40.80791877300004,-73.91901728199997,"{'type': 'Point', 'coordinates': [-73.91901728...",NYC,Crime
3,159828861,2017-01-01T00:00:00.000,744.0,BAIL JUMPING 3,359.0,OFFENSES AGAINST PUBLIC ADMINISTRATION,PL 2155500,M,K,67,...,25-44,M,BLACK,1009039.0,176368.0,40.650739599000076,-73.91066707899995,"{'type': 'Point', 'coordinates': [-73.91066707...",NYC,Crime
4,159834533,2017-01-01T00:00:00.000,203.0,"TRESPASS 3, CRIMINAL",352.0,CRIMINAL TRESPASS,PL 1401000,M,K,60,...,18-24,M,BLACK,986229.0,148311.0,40.573763375000055,-73.99287775699997,"{'type': 'Point', 'coordinates': [-73.99287775...",NYC,Crime


In [28]:
nyc_df['Year'] = pd.DatetimeIndex(nyc_df['Date']).year
nyc_df['Month'] = pd.DatetimeIndex(nyc_df['Date']).month



nyc_df.head()

,Complaint ID,Date,pd_cd,pd_desc,ky_cd,Complaint Description,law_code,Sex,arrest_boro,arrest_precinct,...,Race,x_coord,y_coord,Lat,Lng,lon_lat,NYC,Crime,Year,Month
0,159838726,2017-01-01T00:00:00.000,115.0,RECKLESS ENDANGERMENT 2,355.0,OFFENSES AGAINST THE PERSON,PL 1202000,M,M,14,...,WHITE,988912.0,212647.0,40.750350440000034,-73.98317545899994,"{'type': 'Point', 'coordinates': [-73.98317545...",NYC,Crime,2017,1
1,159824786,2017-01-01T00:00:00.000,101.0,ASSAULT 3,344.0,ASSAULT 3 & RELATED OFFENSES,PL 1200001,M,M,18,...,WHITE,992043.0,217246.0,40.76297132000008,-73.97186979099996,"{'type': 'Point', 'coordinates': [-73.97186979...",NYC,Crime,2017,1
2,159840237,2017-01-01T00:00:00.000,729.0,"FORGERY,ETC.,UNCLASSIFIED-FELONY",113.0,FORGERY,PL 1702500,F,B,40,...,WHITE HISPANIC,1006669.0,233631.0,40.80791877300004,-73.91901728199997,"{'type': 'Point', 'coordinates': [-73.91901728...",NYC,Crime,2017,1
3,159828861,2017-01-01T00:00:00.000,744.0,BAIL JUMPING 3,359.0,OFFENSES AGAINST PUBLIC ADMINISTRATION,PL 2155500,M,K,67,...,BLACK,1009039.0,176368.0,40.650739599000076,-73.91066707899995,"{'type': 'Point', 'coordinates': [-73.91066707...",NYC,Crime,2017,1
4,159834533,2017-01-01T00:00:00.000,203.0,"TRESPASS 3, CRIMINAL",352.0,CRIMINAL TRESPASS,PL 1401000,M,K,60,...,BLACK,986229.0,148311.0,40.573763375000055,-73.99287775699997,"{'type': 'Point', 'coordinates': [-73.99287775...",NYC,Crime,2017,1


In [20]:
# nyc_df.to_csv(f"nyc_data.csv", encoding="utf-8", index=False)

In [ ]:
# x_axis = nyc_df["Year"]
# y_axis = nyc_df["Complaint ID"].value_counts()

# plt.bar(x_axis, y_axis, color= "green")

In [ ]:
# columns =
# nyc_df= nyc_df.rename(columns={
#     'arrest_key': 'Complaint ID',
#     'arrest_date': 'Date',
#     'pd_cd':'Internal Code',
#     'pd_desc':'Internal Description',
#     'ky_cd':'Offence Code',
#     'law_cat_cd':'Offence Level',
#     'law_code':'Offence Law',
#     'ofns_desc':'Description',
#     'arrest_boro':'Borough'}, inplace=True)

##### 